In [1]:
import torch

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

vocoder = torch.hub.load(repo_or_dir='ex3ndr/supervoice-vocoder', model='bigvsan')
vocoder.to(device)
vocoder.eval()

flow = torch.hub.load(repo_or_dir='ex3ndr/supervoice-flow', model='flow', force_reload=True)
flow.to(device)
flow.eval()

Using cache found in /home/steve/.cache/torch/hub/ex3ndr_supervoice-vocoder_master
Downloading: "https://github.com/ex3ndr/supervoice-flow/zipball/main" to /home/steve/.cache/torch/hub/main.zip


AudioFlow(
  (transformer_input): Linear(in_features=200, out_features=1024, bias=True)
  (sinu_pos_emb): LearnedSinusoidalPosEmb()
  (conv_embed): ConvPositionEmbed(
    (dw_conv1d): Sequential(
      (0): Conv1d(1024, 1024, kernel_size=(31,), stride=(1,), padding=(15,), groups=1024)
      (1): GELU(approximate='none')
    )
  )
  (transformer): Transformer(
    (layers): ModuleList(
      (0-23): 24 x AttentionBlock(
        (attention_ln): RMSNorm()
        (attention): Linear(in_features=1024, out_features=3072, bias=False)
        (attention_output): Linear(in_features=1024, out_features=1024, bias=False)
        (mlp_ln): RMSNorm()
        (mlp_input): Linear(in_features=1024, out_features=4096, bias=True)
        (mlp_output): Linear(in_features=4096, out_features=1024, bias=True)
        (mlp_output_dropout): Dropout(p=0.1, inplace=False)
      )
    )
    (skip_combiners): ModuleList(
      (0-11): 12 x Linear(in_features=2048, out_features=1024, bias=True)
    )
    (output_n